In [ ]:
# pip install tqdm
pip install pandastable

In [ ]:
pip install openpyxl

In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk, simpledialog
from pymongo import MongoClient
import threading
import os
import time
import pandas as pd
from pandastable import Table, TableModel  # Make sure to install the pandastable library
import csv
from tqdm import tqdm
import io


In [4]:
# Global variables
client = None
db = None

class UploadThread(threading.Thread):
    def __init__(self, file_path, alert_label, collection, max_upload_time=None):
        super().__init__()
        self.file_path = file_path
        self.alert_label = alert_label
        self.collection = collection
        self.max_upload_time = max_upload_time
        self.start_time = None  # Initialize start_time attribute

    def run(self):
        try:
            df = pd.read_csv(self.file_path, low_memory=False)

            if not df.empty:
                csv_data = df.to_dict(orient='records')
                csv_size = df.memory_usage(deep=True).sum()

                with tqdm(total=csv_size, unit='B', unit_scale=True, desc='Uploading') as pbar:
                    self.start_time = time.time()  # Set start_time when upload begins
                    self.upload_csv(csv_data, pbar)

                self.alert_label.config(text="File uploaded successfully!")
                self.show_popup_message("Upload Complete")
            else:
                messagebox.showerror("Error", "The CSV file is empty.")

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def upload_csv(self, csv_data, pbar):
        for chunk in pd.read_csv(self.file_path, low_memory=False, chunksize=8192):
            size = chunk.memory_usage(deep=True).sum()
            self.collection.insert_many(chunk.to_dict(orient='records'), ordered=False)

            elapsed_time = time.time() - self.start_time
            if self.max_upload_time and elapsed_time > self.max_upload_time:
                break

            upload_speed = size / elapsed_time
            eta = (pbar.total - pbar.n) / upload_speed if upload_speed > 0 else None

            pbar.update(size)

            if eta is not None and eta < 0:
                eta = None

            if eta is not None:
                eta_str = f"{eta:.2f} s" if eta > 0 else "N/A"
            else:
                eta_str = "N/A"

            pbar.set_postfix(
                upload_speed=f"{upload_speed:.2f} B/s",
                eta=eta_str
            )

    def show_popup_message(self, message):
        messagebox.showinfo("Upload Complete", message)
        
def preview_collection_data(selected_database, selected_collection):
    try:
        db = client[selected_database]
        collection = db[selected_collection]

        # Fetch data from MongoDB collection
        cursor = collection.find()
        data = list(cursor)

        # Create a Pandas DataFrame
        df = pd.DataFrame(data)

        # Display the DataFrame using PandasTable
        top = tk.Toplevel()
        top.title(f"Preview - {selected_database}.{selected_collection}")

        frame = tk.Frame(top)
        frame.pack(fill='both', expand=True)

        pt = Table(frame, dataframe=df, showtoolbar=True, showstatusbar=True)
        pt.show()
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while previewing data: {e}")
def convert_excel_to_csv(excel_file_path):
    try:
        xls = pd.ExcelFile(excel_file_path)

        # Iterate over each sheet in the Excel file
        for sheet_name in xls.sheet_names:
            # Read data from the current sheet
            df = pd.read_excel(excel_file_path, sheet_name, dtype=str)

            # Ask the user for the destination directory for the CSV file
            destination_directory = filedialog.askdirectory()
            if not destination_directory:
                return  # User canceled the operation

            # Build the CSV file path including both original filename and sheet name
            csv_file_name = f"{os.path.splitext(os.path.basename(excel_file_path))[0]}_{sheet_name}.csv"
            csv_file_path = os.path.join(destination_directory, csv_file_name)

            # Save the data to a separate CSV file for each sheet
            df.to_csv(csv_file_path, index=False, encoding='utf-8')

            messagebox.showinfo("Conversion Complete", f"Sheet '{sheet_name}' converted and saved to:\n{csv_file_path}")

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred during Excel to CSV conversion: {e}")



# def convert_emt_to_csv_and_display(emt_file_path):
#     try:
#         with open(emt_file_path, 'r') as emt_file:
#             data = emt_file.readlines()

#         # Replace "0.000" with "0" in each line
#         filedata = [item.replace("0.000", "0") for item in data]

#         # Ask the user for the destination directory for the CSV file
#         destination_directory = filedialog.askdirectory()
#         if not destination_directory:
#             return None  # User canceled the operation

#         csv_file_name = os.path.splitext(os.path.basename(emt_file_path))[0] + '_converted.csv'
#         csv_file_path = os.path.join(destination_directory, csv_file_name)

#         with open(csv_file_path, 'w', newline='') as csvfile:
#             csvwriter = csv.writer(csvfile)

#             # Write each row to the CSV file
#             for i in range(len(filedata)):
#                 row = filedata[i].split()
#                 csvwriter.writerow(row)

#         # Display a popup message
#         messagebox.showinfo("Conversion Complete", f"EMT file converted successfully. CSV file saved to:\n{csv_file_path}")

#         return csv_file_path

#     except Exception as e:
#         messagebox.showerror("Error", f"An error occurred during EMT to CSV conversion: {e}")
#         return None

def convert_emt_to_csv_and_display(emt_file_path):
    try:
        with open(emt_file_path, 'r') as emt_file:
            data = emt_file.readlines()

        # Replace "0.000" with "0" in each line
        filedata = [item.replace("0.000", "0") for item in data]

        # Ask the user for the values of "Trial" and "Type" columns
        trial_value = input("Enter value for 'Trial' column: ")
        type_value = input("Enter value for 'Type' column: ")
        subject_name = input("Enter the name of 'Subject Name' column: ")
        cycle_count = input("Enter the No.of 'Cycles' column: ")
        measure_unit = input("Enter value for 'Measure Unit' column: ")

        # Ask the user for the destination directory for the CSV file
        destination_directory = filedialog.askdirectory()
        if not destination_directory:
            return None  # User canceled the operation

        csv_file_name = os.path.splitext(os.path.basename(emt_file_path))[0] + '_converted.csv'
        csv_file_path = os.path.join(destination_directory, csv_file_name)

        with open(csv_file_path, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)

            # Write header row with "Trial" and "Type" columns
            csvwriter.writerow(["Trial", "Type","Subject Name","Cylces","Measure Unit"] + ["Column" + str(i+1) for i in range(len(filedata[0].split()))])

            # Write each row to the CSV file with "Trial" and "Type" values
            for i in range(len(filedata)):
                row = filedata[i].split()
                csvwriter.writerow([trial_value, type_value,subject_name,cycle_count,measure_unit] + row)

        # Display a popup message
        messagebox.showinfo("Conversion Complete", f"EMT file converted successfully. CSV file saved to:\n{csv_file_path}")

        return csv_file_path

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred during EMT to CSV conversion: {e}")
        return None

def create_database(database_entry, database_combobox):
    new_database = database_entry.get()

    if new_database:
        try:
            client[new_database]  # Try to create the database
            messagebox.showinfo("Success", f"Database '{new_database}' created successfully.")
            database_combobox['values'] = client.list_database_names()
            database_combobox.set(new_database)
        except Exception as e:
            messagebox.showerror("Error", f"Unable to create database '{new_database}': {e}")
    else:
        messagebox.showerror("Error", "Please enter a valid database name.")

def create_collection(database_name, collection_name):
    global client
    db = client[database_name]

    try:
        db.create_collection(collection_name)
        messagebox.showinfo("Success", f"Collection '{collection_name}' created successfully.")
    except Exception as e:
        messagebox.showerror("Error", f"Unable to create collection '{collection_name}': {e}")

def connect_to_database(host, port, database_combobox, collection_combobox, database_frame, file_upload_frame):
    try:
        global client, db
        client = MongoClient(f'mongodb://{host}:{port}/', maxPoolSize=10)
        database_names = client.list_database_names()

        if not database_names:
            messagebox.showerror("Error", "No databases found on the server.")
            return

        database_combobox['values'] = database_names
        database_combobox.set(database_names[0] if database_names else '')  # Set default value if available

        database_frame.grid_remove()  # Hide the database connection frame
        file_upload_frame.grid()  # Show the file upload frame

        db = None  # Reset db variable
    except Exception as e:
        messagebox.showerror("Error", f"Unable to connect to the MongoDB server: {e}")

def on_database_change(event, database_combobox, collection_combobox):
    selected_database = database_combobox.get()

    if not selected_database:
        messagebox.showerror("Error", "Please select a database.")
        return

    global db
    db = client[selected_database]

    # Fetch collection names and update the combobox
    collection_names = db.list_collection_names()
    collection_combobox['values'] = collection_names
    collection_combobox.set(collection_names[0] if collection_names else '')  # Set default value if available

def convert_and_upload(file_path_entry, file_type, alert_label, database_combobox, collection_combobox):
    file_path = file_path_entry.get()

    if file_path:
        file_extension = os.path.splitext(file_path)[1].lower()
        selected_database = database_combobox.get()
        selected_collection = collection_combobox.get()

        if not selected_database or not selected_collection:
            messagebox.showerror("Error", "Please select a database and collection.")
            return

        db = client[selected_database]
        collection = db[selected_collection]

        if file_extension == '.csv':
            # Existing code for CSV files (upload)
            upload_thread = UploadThread(file_path, alert_label, collection)
        elif file_extension == '.xlsx':
            # Convert XLSX to CSV without uploading
            csv_file_path = convert_xlsx_to_csv(file_path)
            if csv_file_path:
                print(f"XLSX file converted to CSV: {csv_file_path}")
            else:
                print("Error occurred during XLSX to CSV conversion.")

        elif file_extension == '.emt':
            # New code for EMT files (convert only)
            converted_csv_path = convert_emt_to_csv_and_display(file_path)

            if converted_csv_path:
                print(f"EMT file converted to CSV: {converted_csv_path}")
            else:
                print("EMT to CSV conversion canceled or failed.")
            return  # Do not proceed to upload for EMT files


        # Adding CRUD buttons after starting the upload thread
        crud_buttons_frame = tk.Frame(file_upload_frame)
        crud_buttons_frame.grid(row=11, column=0, pady=5, padx=10, sticky="w")

        create_button = tk.Button(crud_buttons_frame, text="Create Record", command=lambda: create_record(selected_collection))
        create_button.grid(row=0, column=0, pady=5, padx=10, sticky="w")

        read_button = tk.Button(crud_buttons_frame, text="Read Records", command=lambda: read_records(selected_collection))
        read_button.grid(row=0, column=1, pady=5, padx=10, sticky="w")

        update_button = tk.Button(crud_buttons_frame, text="Update Record", command=lambda: update_record(selected_collection))
        update_button.grid(row=0, column=2, pady=5, padx=10, sticky="w")

        delete_button = tk.Button(crud_buttons_frame, text="Delete Record", command=lambda: delete_record(selected_collection))
        delete_button.grid(row=0, column=3, pady=5, padx=10, sticky="w")

        upload_thread.start()

        # Display progress bar during file upload
        upload_progress_bar(upload_thread)
        
def upload_progress_bar(upload_thread):
    top = tk.Toplevel()
    top.title("Upload Progress")

    progress_bar = ttk.Progressbar(top, orient="horizontal", length=300, mode="determinate")
    progress_bar.grid(row=0, column=0, padx=10, pady=10)

    def update_progress():
        while upload_thread.is_alive():
            elapsed_time = time.time() - upload_thread.start_time
            uploading_speed_kb_per_sec = (upload_thread.upload_speed_samples[-1] if upload_thread.upload_speed_samples else 0) / 1024
            progress_bar["value"] = elapsed_time / upload_thread.max_upload_time * 100
            top.update_idletasks()

            upload_speed = upload_thread.upload_speed_samples[-1] if upload_thread.upload_speed_samples else 0
            eta = (progress_bar["maximum"] - progress_bar["value"]) / upload_speed if upload_speed > 0 else None

            progress_bar.set_postfix(
                upload_speed=f"{upload_speed:.2f} B/s",
                eta=f"{eta:.2f} s" if eta is not None else "N/A"
            )

            time.sleep(1)

    progress_thread = threading.Thread(target=update_progress)
    progress_thread.start()

    top.mainloop()
    
# def update_progress(self, pbar):
#     while self.is_alive():
#         current_time = time.time()

#         if self.start_time is None:
#             self.start_time = current_time

#         elapsed_time = current_time - self.start_time

#         if self.max_upload_time and elapsed_time > self.max_upload_time:
#             pbar.close()
#             break

#         try:
#             if (
#                 pbar.n is not None
#                 and elapsed_time > 0
#                 and pbar.total is not None
#                 and pbar.total > 0
#             ):
#                 upload_speed = pbar.n / elapsed_time
#                 eta = (pbar.total - pbar.n) / upload_speed if upload_speed > 0 else None
#             else:
#                 upload_speed = 0
#                 eta = None

#             if (
#                 pbar.n is not None
#                 and eta is not None
#                 and eta > 0
#                 and isinstance(upload_speed, (int, float))
#             ):
#                 pbar.set_postfix(upload_speed=f"{upload_speed:.2f} B/s", eta=f"{eta:.2f} s")
#             elif pbar.n is not None and isinstance(upload_speed, (int, float)):
#                 pbar.set_postfix(upload_speed=f"{upload_speed:.2f} B/s")

#             print(
#                 f"Debug Info: pbar.n={pbar.n}, elapsed_time={elapsed_time}, pbar.total={pbar.total}, "
#                 f"upload_speed={upload_speed}, eta={eta}"
#             )

#         except (ZeroDivisionError, TypeError) as e:
#             print(f"Error: {e}")
#             print(f"pbar.n: {pbar.n}, elapsed_time: {elapsed_time}, pbar.total: {pbar.total}")
#             time.sleep(1)
#             continue

#         time.sleep(1)

def update_progress(self, pbar):
    while self.is_alive():
        try:
            if self.file_size:
                uploaded_size = min(pbar.n, self.file_size)
                remaining_size = max(0, self.file_size - uploaded_size)
                upload_speed = pbar.format_dict['rate'] / 1024.0 if pbar.format_dict['rate'] else 0
                eta = remaining_size / (upload_speed + 1e-9)  # Avoid division by zero
            else:
                upload_speed = 0
                eta = None

            pbar.set_postfix(upload_speed=f"{upload_speed:.2f} KB/s", eta=f"{eta:.2f} s")

        except Exception as e:
            print(f"Error: {e}")
            time.sleep(1)
            continue

        time.sleep(1)

    
def refresh_comboboxes(database_combobox, collection_combobox):
    # Fetch updated database names and update the combobox
    database_names = client.list_database_names()
    database_combobox['values'] = database_names
    database_combobox.set(database_names[0] if database_names else '')  # Set default value if available

    # Clear and update collection combobox
    collection_combobox.set('')
    collection_combobox['values'] = []

def preview_collection_data(selected_database, selected_collection):
    try:
        db = client[selected_database]
        collection = db[selected_collection]

        # Fetch data from MongoDB collection
        cursor = collection.find()
        data = list(cursor)

        # Create a Pandas DataFrame
        df = pd.DataFrame(data)

        # Display the DataFrame using PandasTable
        top = tk.Toplevel()
        top.title(f"Preview - {selected_database}.{selected_collection}")

        frame = tk.Frame(top)
        frame.pack(fill='both', expand=True)

        pt = Table(frame, dataframe=df, showtoolbar=True, showstatusbar=True)
        pt.show()
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while previewing data: {e}")


# Placeholder for CRUD operations
def create_record(collection_name):
    try:
        # Get data from the user
        user_input = simpledialog.askstring("Input", f"Enter data for new record in {collection_name} (comma-separated values):")

        # Convert user input to a dictionary or list (based on your data structure)
        # Example: For simplicity, assuming data is comma-separated and each field is a key-value pair
        data = dict(item.split(":") for item in user_input.split(","))

        # Insert the data into the collection
        db[collection_name].insert_one(data)

        messagebox.showinfo("Success", "Record created successfully.")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while creating a record: {e}")

def read_records(collection_name):
    try:
        # Fetch and display records from the collection
        cursor = db[collection_name].find()
        records = list(cursor)

        print(f"Records in collection {collection_name}:")
        for record in records:
            print(record)
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while reading records: {e}")

def update_record(collection_name):
    try:
        user_input_criteria = simpledialog.askstring("Input", f"Enter update criteria for record in {collection_name} (field:value):")
        update_criteria = dict(item.split(":") for item in user_input_criteria.split(","))

        print("Update Criteria:", update_criteria)

        existing_record = db[collection_name].find_one(update_criteria)
        print("Existing Record:", existing_record)

        if existing_record:
            # Get new data from the user
            user_input_data = simpledialog.askstring("Input", f"Enter new data for the update in {collection_name} (field:value):")
            new_data = dict(item.split(":") for item in user_input_data.split(","))

            # Perform the update
            db[collection_name].update_one(update_criteria, {"$set": new_data})
        else:
            messagebox.showinfo("Info", "Record not found. Nothing updated.")

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while updating a record: {e}")


def delete_record(collection_name):
    try:
        # Get data from the user for the delete
        user_input = simpledialog.askstring("Input", f"Enter delete criteria for record in {collection_name} (field:value):")

        # Convert user input to a dictionary (based on your delete criteria)
        # Example: For simplicity, assuming delete criteria is a single field-value pair
        delete_criteria = dict(item.split(":") for item in user_input.split(","))

        # Check if the record exists before deleting
        existing_record = db[collection_name].find_one(delete_criteria)

        if existing_record:
            # Delete the record from the collection
            db[collection_name].delete_one(delete_criteria)
            messagebox.showinfo("Success", "Record deleted successfully.")
        else:
            messagebox.showinfo("Info", "Record not found. Nothing deleted.")

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while deleting a record: {e}")



def remove_collection(database_name, collection_combobox):
    try:
        # Fetch collection names and update the combobox
        collection_names = client[database_name].list_collection_names()

        if not collection_names:
            messagebox.showinfo("Remove Collection", "No collections to remove.")
            return

        # Create a pop-up window for collection selection
        remove_collection_window = tk.Toplevel()
        remove_collection_window.title("Remove Collection")

        collection_label = tk.Label(remove_collection_window, text="Select Collection:")
        collection_label.grid(row=0, column=0, pady=5, padx=10, sticky="w")

        selected_collection_var = tk.StringVar(value=collection_combobox.get())
        collection_combobox_popup = ttk.Combobox(remove_collection_window, state="readonly", textvariable=selected_collection_var)
        collection_combobox_popup['values'] = collection_names
        collection_combobox_popup.grid(row=0, column=1, pady=5, padx=10, sticky="w")

        confirm_button = tk.Button(remove_collection_window, text="Remove Collection", command=lambda: confirm_remove_collection(database_name, selected_collection_var.get(), collection_combobox, remove_collection_window))
        confirm_button.grid(row=1, column=0, columnspan=2, pady=10)

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while fetching collection names: {e}")

def confirm_remove_collection(database_name, collection_name, collection_combobox, window):
    try:
        # Remove the collection
        client[database_name].drop_collection(collection_name)
        messagebox.showinfo("Remove Collection", f"Collection '{collection_name}' removed successfully.")
        refresh_comboboxes(database_combobox)
        window.destroy()
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while removing collection '{collection_name}': {e}")

def concatenate_excel_files(file_paths, destination_path):
    try:
        # Initialize an empty DataFrame to store concatenated data
        concatenated_df = pd.DataFrame()

        # Iterate over each Excel file
        for excel_file_path in file_paths:
            xls = pd.ExcelFile(excel_file_path)

            # Iterate over each sheet in the Excel file and concatenate data
            for sheet_name in xls.sheet_names:
                df = pd.read_excel(excel_file_path, sheet_name, dtype=str)

                # Add new columns with the source file name and sheet name
                df['SourceFile'] = os.path.basename(excel_file_path)
                df['SheetName'] = sheet_name

                # Concatenate the data
                concatenated_df = pd.concat([concatenated_df, df], ignore_index=True, sort=False)

        # Save the concatenated data to a single CSV file
        concatenated_csv_path = os.path.join(destination_path, f"{os.path.basename(file_paths[0])}_concatenate.csv")
        concatenated_df.to_csv(concatenated_csv_path, index=False, encoding='utf-8')

        messagebox.showinfo("Concatenation Complete", f"All sheets from selected files concatenated and saved to:\n{concatenated_csv_path}")

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred during file concatenation: {e}")


# Example of usage
def concatenate_and_save():
    file_paths = filedialog.askopenfilenames(filetypes=[("Excel files", "*.xlsx")])
    if not file_paths:
        return  # User canceled the operation

    destination_directory = filedialog.askdirectory()
    if not destination_directory:
        return  # User canceled the operation

    concatenate_excel_files(file_paths, destination_directory)
    
# Create the main application window
app = tk.Tk()
app.title("MongoDB GUI")

# Create a Notebook (Tab Control)
notebook = ttk.Notebook(app)
notebook.grid(row=0, column=0, padx=10, pady=10, sticky="w")

# Database connection frame
database_frame = tk.Frame(app)
database_frame.grid(row=0, column=0, pady=10, padx=10, sticky="w")

host_label = tk.Label(database_frame, text="Host:")
host_label.grid(row=0, column=0, pady=5, padx=10, sticky="w")

host_entry = tk.Entry(database_frame, width=20)
host_entry.grid(row=0, column=1, pady=5, padx=10, sticky="w")

port_label = tk.Label(database_frame, text="Port:")
port_label.grid(row=1, column=0, pady=5, padx=10, sticky="w")

port_entry = tk.Entry(database_frame, width=20)
port_entry.grid(row=1, column=1, pady=5, padx=10, sticky="w")

connect_button = tk.Button(database_frame, text="Connect", command=lambda: connect_to_database(host_entry.get(), port_entry.get(), database_combobox, collection_combobox, database_frame, file_upload_frame))
connect_button.grid(row=2, column=4, pady=5, padx=10, sticky="w")

# Database selection frame
file_upload_frame = tk.Frame(notebook)
file_upload_frame.grid(row=0, column=0, padx=10, pady=10, sticky="w")
file_upload_frame.grid_remove()

# Database Combobox
database_combobox_label = tk.Label(file_upload_frame, text="Select Database:")
database_combobox_label.grid(row=2, column=0, pady=5, padx=10, sticky="w")

database_combobox = ttk.Combobox(file_upload_frame, state="readonly")
database_combobox.grid(row=2, column=1, pady=5, padx=10, sticky="w")
database_combobox.bind("<<ComboboxSelected>>", lambda event: on_database_change(event, database_combobox, collection_combobox))

collection_combobox_label = tk.Label(file_upload_frame, text="Select Collection:")
collection_combobox_label.grid(row=3, column=0, pady=5, padx=10, sticky="w")

collection_combobox = ttk.Combobox(file_upload_frame, state="readonly")
collection_combobox.grid(row=3, column=1, pady=5, padx=10, sticky="w")

# File location and upload buttons
file_location_var = tk.StringVar()
file_location_var.set("File Location: ")
file_location_label = tk.Label(file_upload_frame, textvariable=file_location_var, anchor="w", justify="left")
file_location_label.grid(row=4, column=0, pady=5, padx=10, sticky="w")

file_path_entry = tk.Entry(file_upload_frame, width=40)
file_path_entry.grid(row=4, column=1, pady=5, padx=10, sticky="w")

browse_button = tk.Button(file_upload_frame, text="Browse", command=lambda: file_path_entry.insert(0, filedialog.askopenfilename()))
browse_button.grid(row=4, column=2, pady=5, padx=10, sticky="w")

# Database creation frame
database_entry_label = tk.Label(file_upload_frame, text="Create Database:")
database_entry_label.grid(row=0, column=0, pady=5, padx=10, sticky="w")

database_entry = tk.Entry(file_upload_frame, width=20)
database_entry.grid(row=0, column=1, pady=5, padx=10, sticky="w")

create_database_button = tk.Button(file_upload_frame, text="Create Database", command=lambda: create_database(database_entry, database_combobox))
create_database_button.grid(row=0, column=2, pady=5, padx=10, sticky="w")

collection_entry_label = tk.Label(file_upload_frame, text="Create Collection:")
collection_entry_label.grid(row=1, column=0, pady=5, padx=10, sticky="w")

collection_entry = tk.Entry(file_upload_frame, width=20)
collection_entry.grid(row=1, column=1, pady=5, padx=10, sticky="w")

collection_entry_button = tk.Button(file_upload_frame, text="Create Collection", command=lambda: create_collection(database_combobox.get(), collection_entry.get()))
collection_entry_button.grid(row=1, column=2, pady=5, padx=10, sticky="w")

upload_csv_button = tk.Button(file_upload_frame, text="Upload CSV", command=lambda: convert_and_upload(file_path_entry, 'csv',  alert_label,  database_combobox, collection_combobox))
upload_csv_button.grid(row=5, column=0, pady=5, padx=10, sticky="w")

# upload_excel_button = tk.Button(file_upload_frame, text="Convert and Upload Excel", command=lambda: convert_and_upload(file_path_entry, 'excel', time_label, speed_label, alert_label, estimate_label, database_combobox, collection_combobox))
# upload_excel_button.grid(row=5, column=1, pady=5, padx=10, sticky="w")

upload_emt_button = tk.Button(file_upload_frame, text="Convert EMT", command=lambda: convert_and_upload(file_path_entry, 'emt',  alert_label, database_combobox, collection_combobox))
upload_emt_button.grid(row=5, column=2, pady=5, padx=10, sticky="w")

# Preview button
preview_button = tk.Button(file_upload_frame, text="Preview Data", command=lambda: preview_collection_data(database_combobox.get(), collection_combobox.get()))
preview_button.grid(row=3, column=2, pady=5, padx=5, sticky="w")

# Refresh button
refresh_button = tk.Button(file_upload_frame, text="Refresh", command=lambda: refresh_comboboxes(database_combobox, collection_combobox))
refresh_button.grid(row=12, column=5, pady=5, padx=10, sticky="w")

# Labels
# time_label = tk.Label(file_upload_frame, text="Uploading Time: 0.00%")
# time_label.grid(row=6, column=0, pady=5, padx=10, sticky="w")

# speed_label = tk.Label(file_upload_frame, text="Uploading Speed: 0.00 bytes/s (0.00 KB/s)")
# speed_label.grid(row=7, column=0, pady=5, padx=10, sticky="w")

# estimate_label = tk.Label(file_upload_frame, text="Estimated Time Remaining: Calculating...")
# estimate_label.grid(row=8, column=0, pady=5, padx=10, sticky="w")

alert_label = tk.Label(file_upload_frame, text="", fg="green")
alert_label.grid(row=9, column=0, pady=5, padx=10, sticky="w")

# Add a "Concatenate" button
concatenate_button = tk.Button(file_upload_frame, text="Concatenate & convert Excel ", command=concatenate_and_save)
concatenate_button.grid(row=5, column=1, pady=5, padx=10, sticky="w")

# Placeholder for CRUD buttons
crud_buttons_frame = tk.Frame(file_upload_frame)
crud_buttons_frame.grid(row=11, column=0, pady=5, padx=10, sticky="w")

create_button = tk.Button(crud_buttons_frame, text="Create Record", command=lambda: create_record(collection_combobox.get()))
create_button.grid(row=0, column=0, pady=5, padx=10, sticky="w")

read_button = tk.Button(crud_buttons_frame, text="Read Records", command=lambda: read_records(collection_combobox.get()))
read_button.grid(row=0, column=1, pady=5, padx=10, sticky="w")

update_button = tk.Button(crud_buttons_frame, text="Update Record", command=lambda: update_record(collection_combobox.get()))
update_button.grid(row=0, column=2, pady=5, padx=10, sticky="w")

delete_button = tk.Button(crud_buttons_frame, text="Delete Record", command=lambda: delete_record(collection_combobox.get()))
delete_button.grid(row=0, column=3, pady=5, padx=10, sticky="w")

# Remove Collection button
remove_collection_button = tk.Button(file_upload_frame, text="Remove Collection", command=lambda: remove_collection(database_combobox.get(), collection_combobox))
remove_collection_button.grid(row=12, column=2, pady=5, padx=10, sticky="w")

app.mainloop()

Enter value for 'Trial' column: Trial -1
Enter value for 'Type' column: Angle_Data
Enter the name of 'Subject Name' column: Sunil Kumar
Enter the No.of 'Cycles' column: 44
Enter value for 'Measure Unit' column: deg
EMT file converted to CSV: E:/Prasanna/SampleData/Testing_New_EMT_Conversion\A_trial 1_converted.csv
Enter value for 'Trial' column: Trial-1
Enter value for 'Type' column: Force_Data
Enter the name of 'Subject Name' column: Sunil Kumar
Enter the No.of 'Cycles' column: 24
Enter value for 'Measure Unit' column: N
EMT file converted to CSV: E:/Prasanna/SampleData/Testing_New_EMT_Conversion\F_trial 1_converted.csv
Enter value for 'Trial' column: Trial-1
Enter value for 'Type' column: Torque_Data
Enter the name of 'Subject Name' column: Sunil Kumar
Enter the No.of 'Cycles' column: 24
Enter value for 'Measure Unit' column: N*m
EMT file converted to CSV: E:/Prasanna/SampleData/Testing_New_EMT_Conversion\T_trial 1_converted.csv
Enter value for 'Trial' column: Trial-1
Enter value for

Exception in thread Thread-6 (update_progress):
Traceback (most recent call last):
  File "C:\Users\RF04366\AppData\Local\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\RF04366\AppData\Local\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\RF04366\AppData\Local\Temp\ipykernel_1644\2857268976.py", line 317, in update_progress
TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'
Uploading: 100%|██████████████████████████| 108k/108k [00:00<00:00, 10.8MB/s, eta=0.01 s, upload_speed=10760236.44 B/s]
Exception in thread Thread-9 (update_progress):
Traceback (most recent call last):
  File "C:\Users\RF04366\AppData\Local\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\RF04366\AppData\Local\anaconda3\Lib\threading.py", line 975, in run
Uploading:   0%|                                                                           | 0.00

Uploading: 68.1MB [00:02, 27.1MB/s, eta=2.51 s, upload_speed=2294593.77 B/s]                                           
Exception in thread Thread-33 (update_progress):
Traceback (most recent call last):
  File "C:\Users\RF04366\AppData\Local\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\RF04366\AppData\Local\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\RF04366\AppData\Local\Temp\ipykernel_1644\2857268976.py", line 317, in update_progress
TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'
Uploading: 68.4MB [00:02, 27.2MB/s, eta=2.51 s, upload_speed=2309775.34 B/s]                                           
Exception in thread Thread-35 (update_progress):
Traceback (most recent call last):
  File "C:\Users\RF04366\AppData\Local\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\RF04366\AppData\Local\anaconda3\Lib\threadi

In [ ]:
pip install pandastable

In [ ]:
#   File "C:\Users\pugaz\AppData\Local\Temp\ipykernel_10044\1716688761.py", line 529, in <lambda>
#     upload_csv_button = tk.Button(file_upload_frame, text="Upload CSV", command=lambda: convert_and_upload(file_path_entry, 'csv',  alert_label,  database_combobox, collection_combobox))
#   File "C:\Users\pugaz\AppData\Local\Temp\ipykernel_10044\1716688761.py", line 206, in convert_and_upload
#     upload_thread = UploadThread(file_path, alert_label, collection)
# TypeError: __init__() missing 3 required positional arguments: 'alert_label', 'estimate_label', and 'collection'

In [ ]:
def convert_emt_to_csv_and_display(emt_file_path):
    try:
        with open(emt_file_path, 'r') as emt_file:
            data = emt_file.readlines()

        # Replace "0.000" with "0" in each line
        filedata = [item.replace("0.000", "0") for item in data]

        # Ask the user for the values of "Trial" and "Type" columns
        trial_value = input("Enter value for 'Trial' column: ")
        type_value = input("Enter value for 'Type' column: ")

        # Ask the user for the destination directory for the CSV file
        destination_directory = filedialog.askdirectory()
        if not destination_directory:
            return None  # User canceled the operation

        csv_file_name = os.path.splitext(os.path.basename(emt_file_path))[0] + '_converted.csv'
        csv_file_path = os.path.join(destination_directory, csv_file_name)

        with open(csv_file_path, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)

            # Write header row with "Trial" and "Type" columns
            csvwriter.writerow(["Trial", "Type"] + ["Column" + str(i+1) for i in range(len(filedata[0].split()))])

            # Write each row to the CSV file with "Trial" and "Type" values
            for i in range(len(filedata)):
                row = filedata[i].split()
                csvwriter.writerow([trial_value, type_value] + row)

        # Display a popup message
        messagebox.showinfo("Conversion Complete", f"EMT file converted successfully. CSV file saved to:\n{csv_file_path}")

        return csv_file_path

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred during EMT to CSV conversion: {e}")
        return None